# 🧠 Master Data Preparation Notebook — Unified Cleaning, Normalization & Transformation
This notebook integrates all datasets (Sales, Customer, Finance, Employee, Product, Supplier, RAG Docs, KPI) for preprocessing and model training preparation.

In [1]:
# STEP 1: Imports & Setup
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import re, string

# Define storage path
BASE_PATH = 'processed_data'
os.makedirs(BASE_PATH, exist_ok=True)
print('Processed data will be stored in:', BASE_PATH)


Processed data will be stored in: processed_data


In [5]:
# STEP 2: Load all datasets
sales = pd.read_csv('D:/Desktop/GenAI-Powered Analytics Platform/Data/raw/sales_data.csv', parse_dates=['order_date'])
customers = pd.read_csv('D:/Desktop/GenAI-Powered Analytics Platform/Data/raw/customer_data.csv', parse_dates=['signup_date','last_purchase_date'])
finance = pd.read_csv('D:/Desktop/GenAI-Powered Analytics Platform/Data/raw/finance_data.csv', parse_dates=['transaction_date'])
employees = pd.read_csv('D:/Desktop/GenAI-Powered Analytics Platform/Data/raw/employee_data.csv', parse_dates=['joining_date'])
products = pd.read_csv('D:/Desktop/GenAI-Powered Analytics Platform/Data/raw/product_data.csv', parse_dates=['launch_date'])
suppliers = pd.read_csv('D:/Desktop/GenAI-Powered Analytics Platform/Data/raw/supplier_data.csv', parse_dates=['contract_start','contract_end'])
documents = pd.read_csv('D:/Desktop/GenAI-Powered Analytics Platform/Data/raw/business_documents_rag.csv', parse_dates=['created_date'])
kpi = pd.read_csv('D:/Desktop/GenAI-Powered Analytics Platform/Data/raw/daily_kpi_data.csv', parse_dates=['date'])
print('✅ All datasets loaded successfully.')


✅ All datasets loaded successfully.


In [6]:
# STEP 3: Cleaning — Missing values & duplicates
def clean_df(df):
    df = df.drop_duplicates()
    df = df.replace(['None','nan','NaN','NULL','null','?',''], np.nan)
    imputer = SimpleImputer(strategy='most_frequent')
    df[:] = imputer.fit_transform(df)
    return df

sales = clean_df(sales)
customers = clean_df(customers)
finance = clean_df(finance)
employees = clean_df(employees)
products = clean_df(products)
suppliers = clean_df(suppliers)
documents = clean_df(documents)
kpi = clean_df(kpi)
print('✅ Missing values handled and duplicates removed.')


✅ Missing values handled and duplicates removed.


In [7]:
# STEP 4: Data Normalization / Scaling (for numerical columns)
scaler = MinMaxScaler()

def scale_numerical(df):
    num_cols = df.select_dtypes(include=['float64','int64']).columns
    df[num_cols] = scaler.fit_transform(df[num_cols])
    return df

sales = scale_numerical(sales)
customers = scale_numerical(customers)
finance = scale_numerical(finance)
employees = scale_numerical(employees)
products = scale_numerical(products)
suppliers = scale_numerical(suppliers)
kpi = scale_numerical(kpi)
print('✅ Numerical normalization completed.')


✅ Numerical normalization completed.


In [8]:
# STEP 5: Encoding categorical variables
le = LabelEncoder()

def encode_categorical(df):
    cat_cols = df.select_dtypes(include=['object']).columns
    for col in cat_cols:
        try:
            df[col] = le.fit_transform(df[col].astype(str))
        except:
            pass
    return df

sales = encode_categorical(sales)
customers = encode_categorical(customers)
finance = encode_categorical(finance)
employees = encode_categorical(employees)
products = encode_categorical(products)
suppliers = encode_categorical(suppliers)
documents = encode_categorical(documents)
kpi = encode_categorical(kpi)
print('✅ Categorical encoding completed.')


✅ Categorical encoding completed.


In [13]:
# STEP 6: Feature engineering examples (date parts)
def add_date_features(df, date_col):
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        df[date_col + '_year'] = df[date_col].dt.year
        df[date_col + '_month'] = df[date_col].dt.month
        df[date_col + '_day'] = df[date_col].dt.day
        df[date_col + '_weekday'] = df[date_col].dt.weekday
    return df

sales = add_date_features(sales, 'order_date')
finance = add_date_features(finance, 'transaction_date')
employees = add_date_features(employees, 'joining_date')
products = add_date_features(products, 'launch_date')
suppliers = add_date_features(suppliers, 'contract_end')
kpi = add_date_features(kpi, 'date')
print('✅ Date-based features created.')


✅ Date-based features created.


In [16]:
# STEP 7: Merge examples (to build a master dataset)
master = sales.merge(customers, on='customer_id', how='left')\
               .merge(products, on='product_id', how='left', suffixes=('_sale','_prod'))\
               .merge(finance, left_on='region', right_on='department', how='left')

print('Merged master dataset shape:', master.shape)
master.head()


MemoryError: Unable to allocate 14.6 GiB for an array with shape (8, 244557699) and data type float64

In [12]:
# STEP 8: Save processed datasets
sales.to_csv(f'{BASE_PATH}/sales_cleaned.csv', index=False)
customers.to_csv(f'{BASE_PATH}/customer_cleaned.csv', index=False)
finance.to_csv(f'{BASE_PATH}/finance_cleaned.csv', index=False)
employees.to_csv(f'{BASE_PATH}/employee_cleaned.csv', index=False)
products.to_csv(f'{BASE_PATH}/product_cleaned.csv', index=False)
suppliers.to_csv(f'{BASE_PATH}/supplier_cleaned.csv', index=False)
documents.to_csv(f'{BASE_PATH}/documents_cleaned.csv', index=False)
kpi.to_csv(f'{BASE_PATH}/kpi_cleaned.csv', index=False)
#master.to_csv(f'{BASE_PATH}/master_dataset.csv', index=False)
print('✅ All cleaned datasets saved in:', BASE_PATH)


✅ All cleaned datasets saved in: processed_data


## 🚀 Ready for Model Training
- The processed data in `/processed_data/` can now be used for supervised learning (classification/regression) or unsupervised modeling (clustering, segmentation).
- Extend feature generation for domain-specific model inputs (e.g., KPI lag features, text embeddings, etc.).
